# Spark introduction
<br></br>
<center>
<img src="../images/ibm-logo-bw.png" alt="ibm-logo" align="center" style="width: 200px;"/>
</center>

In [1]:
import re
import os
from collections import Counter
import numpy as np
import pandas as pd
from termcolor import cprint
import matplotlib.pyplot as plt

plt.style.use('seaborn')
%matplotlib inline

SMALL_SIZE = 10
MEDIUM_SIZE = 11
LARGE_SIZE = 12

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=LARGE_SIZE)   # fontsize of the figure title

def slide_print(text, color='white'):
    cprint(text, color, 'on_grey')
    
%config IPCompleter.use_jedi = False

Sometimes a single machine simple cannot perform a given task fast enough.  Sometimes there are too many tasks for a single machine to properly handle and yet othertimes there are so much data that the data must be distributed across resources.  These scenarios describe several of the situations where the use of Apache Spark has the potential to directly impact a business opportunity.  Whether you are working with spark locally, on Watson Studio, from within Docker or as part of a computer cluster the basics will cover in this video will still apply.

## High performance computing

* <span style="color:orange">Symmetric multiprocessing</span> -Two or more identical processors connected to a single unit of memory.
* <span style="color:orange">Distributed computing</span> - Processing elements are connected by a network.
* <span style="color:orange">Cluster computing</span> - Group of coupled computers that work together in a way that they can be viewed as a single system.
* <span style="color:orange">Massive parallel processing</span> - Many networked processors usually > 100 used to perform computations in parallel.
* <span style="color:orange">Grid computing</span> - Distributed computing making use of a middle layer to create a `virtual super computer`.

> Spark is a cluster-computing framework. When you compare it to hadoop it essentially competes with which the MapReduce component of the Hadoop ecosystem.

There are many types of high performance computing environments.  READ STATEMENT.  Spark does not have its own distributed filesystem, but can use the Hadoop Distributed File System or HDFS.  Spark uses memory and can use disk for processing, whereas MapReduce has strictly disk-based processing.

### Spark Applications (overview)

<br></br>
<center>
<img src="../images/spark-arch.png" alt="spark-arch" align="center" style="width: 600px;"/>
</center>

> Spark applications run as independent sets of processes on a cluster, coordinated by the SparkContext object in your main program (called the driver program).

Here we have a diagram of Spark Application.  When we start a spark environment a Spark Session is first created and this manages the driver process.  The driver program can be controlled using API's in Scala, Python SQL, Java and R. The worker nodes on the right are usually distinct machines. Executors are the worker nodes' processes, each in charge of running and individual task and they are shown as the orange squares on the worker nodes. One cluster configuration is to assign several cores to each executor leaving one for additional overhead.  The cluster manager, shown on the bottom, which is often YARN, Mesos or Kubernetes helps coordinate between the driver program and worker nodes.

### Spark Applications (general process)

<br></br>
<center>
<img src="../images/spark-arch.png" alt="spark-arch" align="center" style="width: 600px;"/>
</center>

Spark applications are run as an independent sets of processes on a cluster coordinated by the SparkContext object in your main program. Each application gets its own executor processes, which remains allocated for the duration of application.  The driver program, that encapsulates both the SparkContext and the SparkSession is used to submit Spark Applications in Spark .The driver program, once it is given instructions in the form of user code will then ask the cluster manager to launch executors. 

## SparkSession

* SparkContext
* SQLContext
* HiveContext
* StreamingContext

In [2]:
import random

# spark = (ps.sql.SparkSession.builder
#         .appName("sandbox")
#         .getOrCreate()
#         )

# sparkConf = ps.SparkConf()
# sparkConf.setMaster("k8s://https://localhost:6443")
# sparkConf.setAppName("KUBERNETES-IS-AWESOME")
# sparkConf.set("spark.kubernetes.container.image", "robot108/spark-py:latest")
# sparkConf.set("spark.kubernetes.namespace", "playground")

# spark = SparkSession.builder.config(conf=sparkConf).getOrCreate()
# sc = spark.sparkContext

from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
# Create Spark config for our Kubernetes based cluster manager
sparkConf = SparkConf()
sparkConf.setMaster("k8s://https://kubernetes.default.svc.cluster.local:443")
sparkConf.setAppName("spark")
sparkConf.set("spark.kubernetes.container.image", "sbc1/spark-py")
sparkConf.set("spark.kubernetes.namespace", "spark")
sparkConf.set("spark.executor.instances", "1")
sparkConf.set("spark.executor.cores", "1")
sparkConf.set("spark.driver.memory", "512m")
sparkConf.set("spark.executor.memory", "512m")
sparkConf.set("spark.kubernetes.pyspark.pythonVersion", "3")
sparkConf.set("spark.kubernetes.authenticate.driver.serviceAccountName", "spark")
sparkConf.set("spark.kubernetes.authenticate.serviceAccountName", "spark")
sparkConf.set("spark.driver.port", "33333")
sparkConf.set("spark.driver.host", "my-notebook-deployment.spark.svc.cluster.local")
sparkConf.set("spark.kubernetes.executor.volumes.persistentVolumeClaim.my-notebook-pvc.options.claimName","my-notebook-pvc")
sparkConf.set("spark.kubernetes.executor.volumes.persistentVolumeClaim.my-notebook-pvc.mount.path","/root/data")
sparkConf.set("spark.kubernetes.executor.volumes.persistentVolumeClaim.my-notebook-pvc.mount.readOnly","false")
# Initialize our Spark cluster, this will actually
# generate the worker nodes.
spark = SparkSession.builder.config(conf=sparkConf).getOrCreate()
sc = spark.sparkContext


slide_print(spark.version)

for attribute in sc._conf.getAll():
    slide_print(attribute)

2.4.5
('spark.executor.memory', '512m')
('spark.driver.port', '33333')
('spark.kubernetes.executor.volumes.persistentVolumeClaim.my-notebook-pvc.mount.readOnly', 'false')
('spark.kubernetes.authenticate.driver.serviceAccountName', 'spark')
('spark.kubernetes.executor.volumes.persistentVolumeClaim.my-notebook-pvc.mount.path', '/root/data')
('spark.executor.id', 'driver')
('spark.kubernetes.pyspark.pythonVersion', '3')
('spark.driver.memory', '512m')
('spark.kubernetes.namespace', 'spark')
('spark.app.name', 'spark')
('spark.kubernetes.authenticate.serviceAccountName', 'spark')
('spark.rdd.compress', 'True')
('spark.serializer.objectStreamReset', '100')
('spark.executor.instances', '1')
('spark.kubernetes.container.image', 'sbc1/spark-py')
('spark.executor.cores', '1')
('spark.submit.deployMode', 'client')
('spark.master', 'k8s://https://kubernetes.default.svc.cluster.local:443')
('spark.app.id', 'spark-application-1588434508628')
('spark.driver.host', 'my-notebook-deployment.spark.svc.c

Prior to Spark 2.0 there were multiple points of entry for a Spark application, including: SparkContext, SQLContext, HiveContext, and the StreamingContext.  More recent versions of Spark combine all of these objects into a single point of entry that can be used for all Spark applications.  The SparkContext is a child process of the SparkSession. In this cell first we create a SparkSession using the SparkSession builder then we show how to access the Spark Context. Using the SparkContext we show here how to print some of the configuration properties.

## RDD (Resilient Distributed Dataset) 

### An example workflow

1. Create the enviromnent to run spark from python
2. Extract RDDs or DataFrames from files
3. Carry out transformations
4. Execute actions to obtain values (local objects in python)

### The RDD API has two types of operations:

* <span style="color:orange">Transformations</span> - Define a new dataset based on a previous one
* <span style="color:orange">Actions</span> - launch a job for execution on a cluster

In [16]:
text_file = sc.textFile("./sherlock-holmes.txt")
counts = text_file.flatMap(lambda line: line.split(" ")) \
             .map(lambda word: (word, 1)) \
             .reduceByKey(lambda a, b: a + b)
slide_print(type(text_file))  # show that the data are read into an RDD
results = counts.collect()    # output the content in python (CAUTION)
slide_print(results[:7])      # print output

<class 'pyspark.rdd.RDD'>
[('', 3136), ('Project', 81), ('The', 272), ('of', 2723), ('Arthur', 14), ('Conan', 4), ('is', 1074)]


Theses four enumerated steps represent an example workflow.  The code version of these steps are shown just below, with the exception of setting up the environment which we just did.  Spark revolves around the concept of resilient distributed datasets or RDDs.  An RDD is a fault-tolerant collection of elements that can be operated on in parallel. The RDD API, uses two types types of operations: transformations and and actions.  On top of Spark’s RDD API, high level APIs are provided, including the DataFrame API and Machine Learning API, both of which we will cover in this course.  The text_file is an RDD that once subjected to a chain of transformations is used to create counts, another RDD.  The action we use here is 'collect', which brings the data back into Python and it is something you should exercise caution with, especially when working with very large datasets.

### To keep in mind

 * Transformations are the main way of expressing business logic in Spark.
 * Spark doesn't apply the transformation right away, it just builds on the DAG
 * Transformations can be chained together
 *  RDDs are an immutable distributed collection of elements of your data, partitioned across nodes in your cluster that can be operated on in parallel


In [18]:
# an RDD from a an array
rand_nums = np.random.randint(0,500,500000)
rdd = sc.parallelize(rand_nums) 
divis_by_3 = rdd.filter(lambda x: x % 3 == 0)

total1 = rdd.count()
total2 = divis_by_3.count()

print("total divisible by 3: {}/{}".format(total2,total1))

total divisible by 3: 166936/500000


There are several important things to remember about RDDs.  The first is that they use what is known as lazy evaluation.  This means that Spark will wait until the very last moment to execute your transformations.  It does this by constructing a Direct Acyclic Graph or DAG of the transformations.  When an action like count or collect is called the task is sent for execution.  Here we show another example on an RDD, but this time we create it using a numpy array with the parallelize function.  The transformation is a simple filter and the actions are obtain counts rather than all of the data since we generally want to avoid pulling all data into local memory unless we have too.

## Spark-submit

In [25]:
# %%writefile calculate-pi.py

import pyspark as ps
import random

# spark = (ps.sql.SparkSession.builder
#         .appName("get-pi")
#         .getOrCreate()
#         )
# ,
# sc = spark.sparkContext
# random.seed(1)

def sample(p):
    x, y = random.random(), random.random()
    return 1 if x*x + y*y < 1 else 0

rdd = sc.parallelize(range(0, 10000000)) \
           .map(sample) \
           .reduce(lambda a, b: a + b)
count = rdd.count()

result = {"pi": (4.0 * count / 10000000)}
print(result, file=open('calculate-pi-out.txt', 'w'))

AttributeError: 'int' object has no attribute 'count'

```bash
~$ spark-submit calculate-pi.py
```

In [20]:
import ast
with open("calculate-pi-out.txt") as file:  
    results = file.read() 
    results = ast.literal_eval(results)
    
slide_print(results)

{'pi': 3.1414548}


Applications can be submitted to a cluster of any type using the command spark-submit and an accompanying script. The writefile magic function shown here saved the code in the cell as a python script to be called by spark-submit.  The file calculate-pi, unsurprisingly calculates pi, but more importantly it shows an example of how to map a custom function over an RDD.  Spark-submit can be run from the command line as shown.  Generally, it is called with a number of options using a bash file, but it can be called using all the defaults as shown here. Once the script is run it will produce the outfile that is being printed here.  This procedure could be used to make a prediction using a machine learning model that has been tuned, trained and saved. 